In [1]:
# Import Core library
import sys
import re, numpy as np, pandas as pd, matplotlib.pyplot as plt, nltk
import emoji
from pprint import pprint
from cleantext import clean
from bertopic import BERTopic

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


In [2]:
# Import Dataset
df = pd.read_excel('Community306_raw.xlsx')

In [19]:
df.head()

,Unnamed: 0,id,source_id,created_at,geo_source,truncated,text,text_sentiment,text_toxicity,lang,...,in_reply_to_status_id_str,in_reply_to_user_id_str,in_reply_to_screen_name,entities,possibly_sensitive,coordinates,place,create_year,year,month
0,1,1.346170e+18,18,2021-01-04 19:01:23,1,0,@HHere4trump @sbacon0410 #stopthesteal #stopth...,None,None,und,...,1346156649287987201,None,None,"{""entities"": {""hashtags"": [{""end"": 38, ""tag"": ...",0,None,None,2021-01-04 19:01:23,2021,01
1,3,1.345954e+18,18,2021-01-04 04:45:23,None,0,@RudyGiuliani @CaliConserv1 #StoptheSteal #Sto...,None,None,und,...,1345924505483546624,None,None,"{""entities"": {""hashtags"": [{""end"": 41, ""tag"": ...",0,None,None,2021-01-04 04:45:23,2021,01
2,4,1.345954e+18,18,2021-01-04 04:45:13,None,0,#StoptheSteal #StoptheSteal #StoptheSteal #Sto...,None,None,und,...,None,None,None,"{""entities"": {""urls"": [{""end"": 303, ""url"": ""ht...",0,None,None,2021-01-04 04:45:13,2021,01
3,5,1.345952e+18,18,2021-01-04 04:34:34,None,0,#StoptheSteal #StoptheSteal #StoptheSteal #Sto...,None,None,und,...,None,None,None,"{""entities"": {""urls"": [{""end"": 303, ""url"": ""ht...",0,None,None,2021-01-04 04:34:34,2021,01
4,6,1.345825e+18,18,2021-01-03 20:12:15,None,0,@realDonaldTrump @4Libertyinlaw #StoptheSteal ...,None,None,und,...,1345798202650460162,None,None,"{""entities"": {""hashtags"": [{""end"": 45, ""tag"": ...",0,None,None,2021-01-03 20:12:15,2021,01


In [20]:
df['text'].head()

0    @HHere4trump @sbacon0410 #stopthesteal #stopth...
1    @RudyGiuliani @CaliConserv1 #StoptheSteal #Sto...
2    #StoptheSteal #StoptheSteal #StoptheSteal #Sto...
3    #StoptheSteal #StoptheSteal #StoptheSteal #Sto...
4    @realDonaldTrump @4Libertyinlaw #StoptheSteal ...
Name: text, dtype: object

In [21]:
df['create_year'] = pd.to_datetime(df.created_at, format='%d/%m/%Y', errors='coerce')
df['year'] = pd.to_datetime(df.created_at).dt.strftime('%Y')
df['month'] = pd.to_datetime(df.created_at).dt.strftime('%m')

In [22]:
start_date = "15/12/2020"
end_date = "31/01/2021"
after_start_date = df['create_year'] > start_date
before_end_date = df['create_year'] < end_date
between_two_dates = after_start_date & before_end_date

stream_one = df.loc[between_two_dates]

In [23]:
stream_one.head()

,Unnamed: 0,id,source_id,created_at,geo_source,truncated,text,text_sentiment,text_toxicity,lang,...,in_reply_to_status_id_str,in_reply_to_user_id_str,in_reply_to_screen_name,entities,possibly_sensitive,coordinates,place,create_year,year,month
0,1,1.346170e+18,18,2021-01-04 19:01:23,1,0,@HHere4trump @sbacon0410 #stopthesteal #stopth...,None,None,und,...,1346156649287987201,None,None,"{""entities"": {""hashtags"": [{""end"": 38, ""tag"": ...",0,None,None,2021-01-04 19:01:23,2021,01
1,3,1.345954e+18,18,2021-01-04 04:45:23,None,0,@RudyGiuliani @CaliConserv1 #StoptheSteal #Sto...,None,None,und,...,1345924505483546624,None,None,"{""entities"": {""hashtags"": [{""end"": 41, ""tag"": ...",0,None,None,2021-01-04 04:45:23,2021,01
2,4,1.345954e+18,18,2021-01-04 04:45:13,None,0,#StoptheSteal #StoptheSteal #StoptheSteal #Sto...,None,None,und,...,None,None,None,"{""entities"": {""urls"": [{""end"": 303, ""url"": ""ht...",0,None,None,2021-01-04 04:45:13,2021,01
3,5,1.345952e+18,18,2021-01-04 04:34:34,None,0,#StoptheSteal #StoptheSteal #StoptheSteal #Sto...,None,None,und,...,None,None,None,"{""entities"": {""urls"": [{""end"": 303, ""url"": ""ht...",0,None,None,2021-01-04 04:34:34,2021,01
4,6,1.345825e+18,18,2021-01-03 20:12:15,None,0,@realDonaldTrump @4Libertyinlaw #StoptheSteal ...,None,None,und,...,1345798202650460162,None,None,"{""entities"": {""hashtags"": [{""end"": 45, ""tag"": ...",0,None,None,2021-01-03 20:12:15,2021,01


In [24]:
# Convert to list
data = stream_one['text'].values.tolist()

# Make characters lowercase
data = [word.lower() for word in data]

In [25]:
data[:20]

['@hhere4trump @sbacon0410 #stopthesteal #stopthesteal #stopthesteal #stopthesteal #stopthesteal #stopthesteal #stopthesteal #stopthesteal #stopthesteal #stopthesteal #stopthesteal #stopthesteal #stopthesteal #stopthesteal #stopthesteal #stopthesteal #stopthesteal #stopthesteal #stopthesteal #stopthesteal',
 '@rudygiuliani @caliconserv1 #stopthesteal #stopthesteal #stopthesteal #stopthesteal #stopthesteal #stopthesteal #stopthesteal #stopthesteal #stopthesteal #stopthesteal #stopthesteal #stopthesteal #stopthesteal #stopthesteal #stopthesteal #stopthesteal #stopthesteal #stopthesteal #stopthesteal #stopthesteal',
 '#stopthesteal #stopthesteal #stopthesteal #stopthesteal #stopthesteal #stopthesteal #stopthesteal #stopthesteal #stopthesteal #stopthesteal #stopthesteal #stopthesteal #stopthesteal #stopthesteal #stopthesteal #stopthesteal #stopthesteal #stopthesteal #stopthesteal #stopthesteal https://t.co/2fgak2rhcd',
 '#stopthesteal #stopthesteal #stopthesteal #stopthesteal #stopthesteal

In [39]:
def get_url_patern():
    return re.compile(
        r'(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))'
        r'[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9]\.[^\s]{2,})')

def get_hashtags_pattern():
    return re.compile(r'#\w*')

def get_single_letter_words_pattern():
    return re.compile(r'(?<![\w\-])\w(?![\w\-])')

def get_blank_spaces_pattern():
    return re.compile(r'\s{2,}|\t')

def get_twitter_reserved_words_pattern():
    return re.compile(r'(RT|rt|FAV|fav|VIA|via)')

def get_mentions_pattern():
    return re.compile(r'@\w*')

def process_text(word):
  word=re.sub(pattern=get_url_patern(), repl="", string=word)
  word=re.sub(pattern=get_mentions_pattern(), repl="", string=word)
  word=re.sub(pattern=get_hashtags_pattern(), repl="", string=word)
  word=re.sub(pattern=get_twitter_reserved_words_pattern(), repl='', string=word)
#   word=re.sub(pattern=get_single_letter_words_pattern(), repl='', string=word)
#   word=re.sub(pattern=get_blank_spaces_pattern(), repl=' ', string=word)
#   word=re.sub('\s+', " ", word)
#   word=re.sub("\'", "", word)
#   word=re.sub(r'http\S+', "", word)  # remove http links
#   word=re.sub(r'bit.ly/\S+', "", word)  # rempve bitly links
#   word=word.strip('[link]')  # remove [links]
  word=re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', "", word)  # remove retweet
  word=re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', "", word)  # remove tweeted at
  word=clean(word, no_emoji=True)
#   word =re.sub(r'\d+', '', word)
#   word = word.strip()
  return word

In [42]:
data = [process_text(word) for word in data]

In [43]:
topic_model = BERTopic()
topics, probs = topic_model.fit_transform(data)

In [44]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,14683,-1_he_what_that_be
1,0,12602,0_awome_hooah_yuge_hello
2,1,1598,1_note_every__
3,2,1248,2_cap_location_inciter_minutes
4,3,1181,3_flight_kicked_wearing_joke
...,...,...,...
963,959,10,959_help_ging_ty_please
962,964,10,964_bye_byebye_goodbye_riddance
961,965,10,965_fraudulent_deceify_cancel_produced
959,963,10,963_visits_outmilitary_special_authorised


In [45]:
topic_model.get_topic(0)

[('awome', 0.6912742820493176),
 ('hooah', 0.5816118604050453),
 ('yuge', 0.4840242308167575),
 ('hello', 0.39357395320454625),
 ('ron', 0.33634570536793085),
 ('oh', 0.1738989433247991),
 ('hey', 0.15427970608475794),
 ('man', 0.11743627758280287),
 ('their', 0.042134393673152165),
 ('that', 0.014966125694695754)]

In [46]:
topic_model.get_topic(1)

[('note', 1.643147404435563),
 ('every', 0.42864320913535475),
 ('', 1e-05),
 ('', 1e-05),
 ('', 1e-05),
 ('', 1e-05),
 ('', 1e-05),
 ('', 1e-05),
 ('', 1e-05),
 ('', 1e-05)]

In [47]:
topic_model.get_topic(2)

[('cap', 0.04520916311789243),
 ('location', 0.04485752425703519),
 ('inciter', 0.04450256363692796),
 ('minutes', 0.044476187067344075),
 ('tweeted', 0.04392960599910656),
 ('three', 0.0431227394922374),
 ('nancy', 0.04308314632032886),
 ('stormed', 0.042108741038176294),
 ('terrorists', 0.04134566235916539),
 ('after', 0.03362789769054032)]

In [48]:
topic_model.get_topic(3)

[('flight', 0.02961639225602412),
 ('kicked', 0.0296107115177797),
 ('wearing', 0.029498344367083373),
 ('joke', 0.028150581756905175),
 ('mask', 0.027669616559395822),
 ('suppoer', 0.02646867824331035),
 ('made', 0.02564814124999657),
 ('she', 0.023977157884988814),
 ('off', 0.02321231408252666),
 ('got', 0.022619733941439436)]

In [49]:
topic_model.get_topic(4)

[('prerally', 0.06937586993053489),
 ('plaza', 0.06695246199741428),
 ('freedom', 0.05254254146269504),
 ('coalition', 0.04547502443109604),
 ('live', 0.03156073513633657),
 ('now', 0.022429141002735437),
 ('at', 0.01451769476215901),
 ('dc', 0.013184112747835972),
 ('in', 0.006617577691502449),
 ('ve', 0.0006751588457546932)]

In [25]:
topic_model.get_topic(0)

[('diot', 0.8623393167912485),
 ('hooah', 0.7252647969221329),
 ('cymi', 0.6616631030905615),
 ('sickening', 0.5176435785524879),
 ('hello', 0.4902466670351643),
 ('desperate', 0.26834738748387604),
 ('cou', 0.1859258891833013),
 ('you', 0.008560844086377969),
 ('', 1e-05),
 ('', 1e-05)]

In [50]:
topic_model.visualize_barchart()

In [67]:
stream_one['create_year'].head()
timestamps = stream_one.create_year.to_list()
tweets = data
timestamps[:5]

[Timestamp('2021-01-04 19:01:23'),
 Timestamp('2021-01-04 04:45:23'),
 Timestamp('2021-01-04 04:45:13'),
 Timestamp('2021-01-04 04:34:34'),
 Timestamp('2021-01-03 20:12:15')]

In [69]:
topics_over_time = topic_model.topics_over_time(tweets, topics, timestamps, nr_bins=20)

In [70]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=6)